In [1]:
import pickle
with open('data_full_0413.pkl','rb') as f:
    data = pickle.load(f)
data.shape

(2918, 229)

In [23]:
data_test_origin = data[data['training_set'] == False]
data_test_origin.shape

(1459, 229)

In [49]:
with open('data_train_0413.pkl','rb') as f:
    data_train = pickle.load(f)
data_train.shape

(1459, 227)

In [3]:
with open('data_test_0413.pkl','rb') as f:
    data_test = pickle.load(f)
data_test.shape

(1459, 227)

In [7]:
with open('target_0413.pkl','rb') as f:
    target = pickle.load(f)
target.shape

(1459,)

In [13]:
import xgboost as xgb
xgb_regressor = xgb.XGBRegressor(random_state=42) #用于交叉验证
#假设已有最好参数
xgb_opt = xgb.XGBRegressor(learning_rate=0.05,max_depth=5,min_child_weight=1.5,n_estimators=7500,seed=42)
xgb_opt.fit(data_train,target)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=5, min_child_weight=1.5, missing=None, n_estimators=7500,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=1)

In [16]:
from sklearn.metrics import r2_score, make_scorer, mean_squared_error
xgb_opt_predict = xgb_opt.predict(data_train)
xgb_opt_mse = mean_squared_error(target,xgb_opt_predict)
xgb_opt_mse

5.197345312176011e-06

In [24]:
import numpy as np
import pandas as pd
y_pred = xgb_opt.predict(data_test)
y_pred_final = np.exp(y_pred)
my_submission = pd.DataFrame({'Id':data_test_origin.Id,'SalePrice':y_pred_final})
my_submission.to_csv('submission_0413_xgb.csv',index=False)

## 将重要特征取log，试试data_train

In [50]:
data_train = pd.concat([data_train,target],axis=1)

In [51]:
data_train.shape

(1459, 228)

In [52]:
#删除可能噪音
data_train.sort_values(by = 'GrLivArea',ascending=False)[:2]

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleType_None,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SalePrice
1298,60,63887,10,5,2008,2008,5644.0,0.0,466.0,6110.0,...,0,0,0,0,0,0,0,0,1,11.982929
523,60,40094,10,5,2007,2008,2260.0,0.0,878.0,3138.0,...,0,0,0,0,0,0,0,0,1,12.126759


In [53]:
data_train = data_train.drop(index=1298,axis=0)

In [54]:
data_train = data_train.drop(index=523,axis=0)

In [55]:
#取log看看
data_train['GrLivArea'] = np.log(data_train['GrLivArea'])

In [56]:
#0值不能取log，新创建1列，if area>0,gets 1;if area=0,gets 0
data_train['HasBsmt'] = 0
data_train.loc[data_train['TotalBsmtSF']>0,'HasBsmt'] = 1
data_train.loc[data_train['HasBsmt']==1,'TotalBsmtSF'] = np.log(data_train['TotalBsmtSF'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [57]:
target1 =data_train['SalePrice'] 

In [58]:
data_train = data_train.drop('SalePrice',axis=1)

## 将重要特征取log，再试data_test

In [ ]:
#取log看看
data_test['GrLivArea'] = np.log(data_test['GrLivArea'])

In [31]:
#0值不能取log，新创建1列，if area>0,gets 1;if area=0,gets 0
data_test['HasBsmt'] = 0
data_test.loc[data_test['TotalBsmtSF']>0,'HasBsmt'] = 1
data_test.loc[data_test['HasBsmt']==1,'TotalBsmtSF'] = np.log(data_test['TotalBsmtSF'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


## xgboost 建模训练

In [60]:
xgb_regressor = xgb.XGBRegressor(random_state=42) #用于交叉验证
#假设已有最好参数
xgb_opt1 = xgb.XGBRegressor(learning_rate=0.05,max_depth=5,min_child_weight=1.5,n_estimators=7500,seed=42)
xgb_opt1.fit(data_train,target1)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=5, min_child_weight=1.5, missing=None, n_estimators=7500,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=1)

In [62]:
xgb_opt_predict = xgb_opt1.predict(data_train)
xgb_opt_mse = mean_squared_error(target1,xgb_opt_predict)
xgb_opt_mse

4.969462919730817e-06

## 使用网格搜索交叉验证，寻找最佳参数

In [67]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
xgb_regressor = xgb.XGBRegressor(random_state=42) #用于交叉验证
parameters_xgb = {'n_estimators':[7000,7500,8000],\
                 'learning_rate':[0.03,0.05,0.07],\
                 'max_depth':[3,5,7],\
                 'min_child_weight':[1,1.5,2]}
grid_obj_xgb = RandomizedSearchCV(xgb_regressor,parameters_xgb,random_state=99)
grid_fit_xgb = grid_obj_xgb.fit(data_train,target1)
xgb_opt_final = grid_fit_xgb.best_estimator_

In [68]:
xgb_opt_final

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=5, min_child_weight=2, missing=None, n_estimators=7000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [69]:
xgb_opt_predict = xgb_opt_final.predict(data_train)
xgb_opt_mse = mean_squared_error(target1,xgb_opt_predict)
xgb_opt_mse

4.969462919730817e-06

In [ ]:
y_pred = rf_opt.predict(data_test.values)
y_pred_final = np.exp(y_pred)

In [ ]:
my_submission = pd.DataFrame({'Id':data_test.Id,'SalePrice':y_pred_final})
my_submission.to_csv('submission_0413_xgbt.csv',index=False)